In [2]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:45151,Cluster Workers: 2 Cores: 2 Memory: 33.39 GB


In [ ]:
import os
import urllib.request

station_meta_url = 'ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv'
base_url = 'ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/by_year/'

if not os.path.isfile('/data/weather/isd-history.csv'):
    urllib.request.urlretrieve(station_meta_url, '/data/weather/isd-history.csv')

years = list(range(2000, 2018))
for year in years:
    fn = str(year) + '.csv.gz'
    if not os.path.isfile('/data/weather/'+fn):
        urllib.request.urlretrieve(base_url+fn, '/data/weather/'+fn)

In [37]:
import dask_cudf

names = ["id", "date", "type", "val", "time", "dummy1", "dummy2", "dummy3"]
dtype = ["int" for name in names]
usecols = names[0:4]

decade = '2000'
files = '/data/weather/'+decade+'/*.csv.gz'
df = dask_cudf.read_csv(files, names=names, dtype=dtype, usecols=usecols)

In [38]:
df.head().to_pandas()

,id,date,type,val
0,2147483647,20000101,39110,350
1,2147483647,20000101,35622,0
2,2147483647,20000101,39110,141
3,2147483647,20000101,39180,79
4,2147483647,20000101,35622,10


In [41]:
len(df)

343314695

In [39]:
%time df.groupby(by=["type"]).count().compute()

CPU times: user 1.34 s, sys: 229 ms, total: 1.57 s
Wall time: 19.4 s


<cudf.DataFrame ncols=4 nrows=107 >

In [40]:
%time df.groupby(by=["id"]).count().compute()

CPU times: user 1.34 s, sys: 182 ms, total: 1.52 s
Wall time: 19.8 s


<cudf.DataFrame ncols=4 nrows=1 >

In [29]:
print(df.groupby(by=["id"]).().compute())

    type count_id count_date count_val
 0 38023     1742   17306010    118896
 1 39001    76802   14911416    224974
 2 39021     2090      98565      2554
 3 42610      437    5937475     45182
 4 38031    24431     533338    237062
 5 22090     6937     225084    176665
 6 38011     3092      17190    476400
 7 39002   451314     360711     81407
 8 42601  1094908      82196     73377
 9 32350   658770     239792      3457
[88 more rows]


In [7]:
@delayed
def load_file(fn, names, dtypes, skiprows):
    return cudf.read_csv(fn, names=names, dtype=dtypes, skiprows=skiprows)

years = [2017, 2018]
files = ['/data/weather/'+str(year)+'.csv' for year in years]

# csv columns
names = ["id", "date", "type", "val", "time", "dummy1", "dummy2", "dummy3"]
dtypes = ["int" for name in names]
# use workers to read files
weather_df = dask_cudf.from_delayed([load_file(fn, names, dtypes, 1) for fn in files])

In [10]:
print(weather_df.head())

          id     date  type  val time dummy1 dummy2 dummy3
0 2147483647 20170101 38349    0                 30       
1 2147483647 20170101 38410    0                 30       
2 2147483647 20170101 35622    0                 30       
3 2147483647 20170101 39110  274                 49       
4 2147483647 20170101 39180  218                 49       


In [27]:
names = ["USAF","WBAN","STATION NAME","CTRY","STATE","ICAO","LAT","LON","ELEV(M)","BEGIN","END"]
dtypes = ['int' for col in names]
df = cudf.read_csv('/data/weather/isd-history.csv', names=names, dtype=dtypes, skiprows=1)

In [26]:
print(sample.val)

      
0    0
1    0
2    0
3    0
4    0


In [9]:
files = ['test1.csv', 'test2.csv']
lines = ['num0,num1', '0,1', '2,3']

for file in files:
    with open(file, 'w') as fp:
        fp.write('\n'.join(lines)+ '\n')

@delayed
def load_file(fn, names, dtypes, skiprows):
    return cudf.read_csv(fn, names=names, dtype=dtypes, skiprows=skiprows)

# csv columns
names = ["num0", "num1"]
dtypes = ["int" for name in names]
df = dask_cudf.from_delayed([load_file(fn, names, dtypes, 1) for fn in files])

In [9]:
print(df.head())

NameError: name 'df' is not defined

In [ ]:
from cudf import DataFrame as DF, Series as S
from numpy.random import randint
x = DF({'x': S(randint(0, 2**63, size=5)).astype('datetime64[ms]')})
DF.from_arrow(x.to_arrow())